## Importing relevant libraries

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Constructing the Schlogl operator

In [3]:
def TridiagA(lambda_func, mu_func, N):
    """
    Creates tridiagonal stochastic matrix for the Schlogl model
    Inputs lambda and mu model functions, and N desired size of matrix
    Returns stochastic matrix (non-hermitian)
    """
    # initialize diagonals
    d1 = np.zeros(N - 1)
    d2 = np.zeros(N)
    d3 = np.zeros(N - 1)

    # element 1,1
    d2[0] = -lambda_func(0)

    # element N,N
    d2[N - 1] = -mu_func(N - 1)

    # bottom diagonal elements
    for i in range(N - 1):
        d1[i] = lambda_func(i)

    # top diagonal elements
    for i in range(1, N):
        d3[i - 1] = mu_func(i)

    # main diagonal elements
    for i in range(1, N - 1):
        d2[i] = -lambda_func(i) - mu_func(i)

    # putting the diagonals together
    Q = np.diag(d1, k = -1) + np.diag(d2, k = 0) + np.diag(d3, k = 1)
    
    return Q

def get_volume_array(start_V, stop_V, n_operator_qubits):
    # Function to generate the initial volume array (to carry out computations)
    
    num_elements = 2 ** n_operator_qubits
    step_size = (stop_V - start_V)/(num_elements - 1)
    
    # Generate the volume array, given that the step size has been determined
    volume_array = np.arange(start_V, stop_V, step_size)
    
    return volume_array

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)
    
    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes

    # Get absolute values
    b = np.abs(b)
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_rmsd(list1, list2):
    # Ensure the lists have the same length
    
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length.")
    
    # Compute the RMSD
    rmsd = np.sqrt(np.mean((np.array(list1) - np.array(list2)) ** 2))
    return rmsd

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_eigenvalues(matrix):
    """
    Computes the two eigenvalues with the highest real parts of a general matrix.

    Parameters:
        matrix (np.ndarray): The matrix for which to compute the eigenvalues.

    Returns:
        list: A list containing the two eigenvalues with the highest real parts [eigenvalue1, eigenvalue2].
    """
    # Compute all eigenvalues
    eigenvalues = np.linalg.eigvals(matrix)
    
    # Sort by real part in descending order and select the two with the lowest real parts
    lowest_two_eigenvalues = sorted(eigenvalues, key=lambda x: x.real, reverse=False)[:2]

    # Print
    print('The lowest two eigenvalues are:')
    print(lowest_two_eigenvalues)
    print()
    
    return lowest_two_eigenvalues

def compute_relative_errors(exact_eigenvalues, vqd_eigenvalues):
    """
    Computes the relative errors for the first and second eigenvalues for each optimizer.

    Parameters:
        exact_eigenvalues (list): List of the two lowest exact eigenvalues [lambda_1, lambda_2].
        vqd_eigenvalues (dict): Dictionary where keys are optimizer names and values are lists of the two
                                lowest VQD eigenvalues computed by each optimizer.

    Returns:
        dict: Dictionary where each key is an optimizer name and the value is a tuple of the relative errors
              (error_first, error_second).
    """
    # Extract the exact eigenvalues for easy reference
    exact_first, exact_second = exact_eigenvalues
    
    # Initialize a dictionary to store relative errors for each optimizer
    relative_errors = {}
    
    for optimizer, eigenvalues in vqd_eigenvalues.items():
        vqd_first, vqd_second = eigenvalues  # VQD-computed first and second eigenvalues for this optimizer
        
        # Calculate relative errors for the first and second eigenvalues
        error_first = abs((vqd_first - exact_first) / exact_first) if exact_first != 0 else np.nan
        error_second = abs((vqd_second - exact_second) / exact_second) if exact_second != 0 else np.nan
        
        # Store the errors in the dictionary
        relative_errors[optimizer] = (error_first, error_second)
    
    return relative_errors

## VQD implementation

In [6]:
# Define the VQD computation with varying ansatz depth
def run_vqd(matrix, ansatz, optimizer, seed, zeromode_exact):
    # Function to compute the execution time for different optimizers and ansatz depths (averaged over 10 independent VQD runs in each case)

    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the qubit Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Compute using NumPyMinimumEigensolver for reference
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qub_hamiltonian)
    
    # Initial point for the classical optimizer
    seed_value = seed
    np.random.seed(seed_value)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initializing the Estimator, Sampler, and fidelity
    estimator = Estimator()
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler)
    
    # Run the VQE algorithm
    @dataclass
    class VQDLog:
        values: list = None
        parameters: list = None
        
        def __post_init__(self):
            self.values = []
            self.parameters = []

        # Update function to match the expected arguments
        def update(self, count, parameters, mean, _metadata, _extra):
            self.values.append(mean)
            self.parameters.append(parameters)

    log = VQDLog()

    vqd = VQD(estimator,
              fidelity, ansatz, optimizer, k=2, initial_point=initial_point, callback=log.update)
    result = vqd.compute_eigenvalues(qub_hamiltonian)

    # Estimate the zeromode
    optimal_params = result.optimal_points
    zeromode_points = optimal_params[0]
    final_circuit = ansatz.assign_parameters(zeromode_points)
    zeromode_vqd = Statevector.from_instruction(final_circuit)
    zeromode = zeromode_vqd.data.tolist()

    # Get the eigenvalues
    eigenvalues_vqd = result.eigenvalues

    # Compute the fidelity between the classical and quantum zeromodes
    zeromode_exact = np.array(zeromode_exact).reshape(len(zeromode_exact), 1)
    zeromode = np.array(zeromode).reshape(len(zeromode_exact), 1)
    fidelity = get_similarity(zeromode_exact, zeromode)

    # Get the number of function calls
    num_func_calls = result.cost_function_evals

    return zeromode, fidelity, eigenvalues_vqd, num_func_calls

## Evaluating performance metrics at multiple ansatz depths

In [9]:
def analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth):

    # Initialize dictionaries to store results
    zeromodes = {}
    metrics = {}  # To store fidelity and function call counts
    eigenvalues = {} # to store the eigenvalues

    optimizer_name = optimizer.__class__.__name__

    # Loop through each ansatz type
    for AnsatzClass in [RealAmplitudes, TwoLocal, EfficientSU2]:
        # Initialize ansatz with appropriate configurations
        if AnsatzClass == RealAmplitudes:
            ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=1)
        elif AnsatzClass == TwoLocal:
            ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'], entanglement_blocks='cx', reps=1)
        elif AnsatzClass == EfficientSU2:
            ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'], entanglement='sca', reps=1)

        ansatz_name = AnsatzClass.__name__
        pair_name = f"{optimizer_name}-{ansatz_name}"
        print(f"\nRunning VQD for optimizer-ansatz pair: {pair_name}")

        # Initialize dictionaries to store depth-specific results
        zeromodes[pair_name] = {}
        metrics[pair_name] = {}
        eigenvalues[pair_name] = {}

        for current_depth in range(1, max_depth + 1):
            # Set `reps` (depth) for the current ansatz
            ansatz.reps = current_depth

            # Temporary storage for the fidelity results
            fidelities = []
            function_calls = []  # To track the number of function calls for each run
            all_zeromodes = []  # To store all zeromodes for the depth
            all_eigenvalues = [] # to store all the eigenvalues

            # Perform multiple independent VQE runs to calculate fidelity and resource usage
            for run in range(10):  # Number of independent runs
                seed = run + 1  # Set a unique seed for each run

                # Run VQD
                zeromode, fidelity_value, eigenvalues_vqd, function_call_count = run_vqd(
                    matrix=matrix, ansatz=ansatz, optimizer=optimizer, seed=seed,
                    zeromode_exact=zeromode_classic)

                # Append results
                fidelities.append(fidelity_value)
                function_calls.append(function_call_count)
                all_zeromodes.append(zeromode)
                all_eigenvalues.append(eigenvalues_vqd)

            # Identify the run with the highest fidelity at this depth
            best_run_index = np.argmax(fidelities)
            best_fidelity = fidelities[best_run_index]
            best_zeromode = all_zeromodes[best_run_index]
            best_function_calls = function_calls[best_run_index]
            best_eigenvalues = all_eigenvalues[best_run_index]

            # Store results for the current depth
            zeromodes[pair_name][current_depth] = best_zeromode
            metrics[pair_name][current_depth] = {
                'fidelity': best_fidelity,
                'function_calls': best_function_calls
            }
            eigenvalues[pair_name][current_depth] = {
                'lowest eigenvalue': best_eigenvalues[0], 
                'first excited state eigenvalue': best_eigenvalues[1]
            }

            print(f"{pair_name} - Depth {current_depth}: Best fidelity = {best_fidelity}, Function calls = {best_function_calls}, \
            Best zeromode: {best_zeromode}, Best lowest eigenvalue: {best_eigenvalues[0]}, First excited state eigenvalue: {best_eigenvalues[1]}")

    return zeromodes, metrics, eigenvalues

## Compute the matrix, zeromode, and eigenvalues

In [12]:
def get_zeromode(num_operator_qubits, V):
    # Function to get the matrix and the zeromode

    ## Computing the block diagonal representation of the Schlogl operator matrix
    # Defining parameters
    a = 1
    b = 1
    k1 = 3
    k2 = 0.6
    k3 = 0.25
    k4 = 2.95
    
    # Number of qubits
    num_operator_qubits = num_operator_qubits
    
    # Matrix dimensions
    N = 2 ** num_operator_qubits
    
    # Generating the basis size array
    x_axis = []
    
    for i in range(N):
        x_axis.append(i)
    
    # # Constructing the Schlogl operator for V = 1.1
    # # Get the volume array
    # start_V = 0.1
    # stop_V = 1.6
    # volume_array = get_volume_array(start_V, stop_V, num_operator_qubits)
    
    # # For system volume V = 1.1
    # volume_array = np.arange(0.1, 10.6, 0.1)
    
    # # Construct the matrix representation of the operator
    # for i, V in enumerate(volume_array):
        
    #     # Birth and death rates
    #     lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    #     mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)
    
    #     # stochastic matrix Q of dimension N x N
    #     Q = TridiagA(lambda_fn, mu_fn, N)
        
    #     i += 1
    # ######################################################################  
    # Construct the matrix
    # Birth and death rates
    lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)

    # stochastic matrix Q of dimension N x N
    Q = TridiagA(lambda_fn, mu_fn, N)
    
    # Print the original Schlogl operator matrix
    print('The Schlogl operator matrix is:')
    print(Q)
    print()
        
    # Compute the Hermitian form of the matrix
    hermitian_matrix = np.dot(Q.T, Q)
    
    # # Print the volume array
    # print('The volume array is:')
    # print(volume_array)
    # print()
    
    # Print the volume parameter for which the simulations are being run
    print('The volume parameter is:')
    print(V)
    print()
    
    # Print the Hermitian matrix (block diagonal form)
    print('The Hermitian form of the Schlogl matrix is:')
    print(hermitian_matrix)
    print()

    # Get the lowest two eigenvalues
    lowest_eigenvalues = compute_eigenvalues(hermitian_matrix)
    
    ## Get the classical zeromode
    A, P = la.eig(Q)
    
    A_real = np.real(A)
    index = np.where(A_real == np.amax(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))
    
    print("the available eigenvalues are: \n" + str(A))
    print()
    
    print("The minimum eigenvalue located is: \n" + str(eigenvalue))
    print()
    
    print("The minimum zeromode located is: \n" + str(np.real(zeromode_classic)))
    print()

    print("The lowest two eigenvalues are: \n" + str(np.real(lowest_eigenvalues)))
    print()

    return Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues

## Quantum (VQD) analysis

### For V = 1.1

### For 2 qubits

### Get the matrix, zeromode, and eigenvalues

In [21]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(2, 1.1)

The Schlogl operator matrix is:
[[ -0.275        2.95         0.           0.        ]
 [  0.275       -3.225        5.9          0.        ]
 [  0.           0.275      -11.62954545  11.82520661]
 [  0.           0.           5.72954545 -11.82520661]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix is:
[[ 1.51250000e-01 -1.69812500e+00  1.62250000e+00  0.00000000e+00]
 [-1.69812500e+00  1.91787500e+01 -2.22256250e+01  3.25193182e+00]
 [ 1.62250000e+00 -2.22256250e+01  2.02884019e+02 -2.05274837e+02]
 [ 0.00000000e+00  3.25193182e+00 -2.05274837e+02  2.79671023e+02]]

The lowest two eigenvalues are:
[1.0754013022257892e-14, 9.002123036682548]

the available eigenvalues are: 
[  0.        +0.j  -2.60510311+0.j  -4.34244163+0.j -20.00720732+0.j]

The minimum eigenvalue located is: 
0j

The minimum zeromode located is: 
[[0.99567159]
 [0.09281684]
 [0.00432621]
 [0.00209613]]

The lowest two eigenvalues are: 
[1.07540130e-14 9.00212304e+00]



### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [34]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2, eigenvalues_N_2 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = [135 365],             Best zeromode: [[-0.9956716 +0.j]
 [-0.09281682+0.j]
 [-0.00432616+0.j]
 [-0.00209605+0.j]], Best lowest eigenvalue: (7.993605777301127e-13+0j), First excited state eigenvalue: (9.002123036682553+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = [319 253],             Best zeromode: [[0.9956716 +0.j]
 [0.09281676+0.j]
 [0.00432616+0.j]
 [0.0020961 +0.j]], Best lowest eigenvalue: (6.927791673660977e-14+0j), First excited state eigenvalue: (9.002123036682422+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [169 221],             Best zeromode: [[0.99567159+0.j]
 [0.09281691+0.j]
 [0.00432625+0.j]
 [0.00209615+0.j]], Best lowest eigenvalue: (7.815970093361102e-14+0j), First excited state eigenvalue: (9.002123036682862+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [225 735],             Best zeromode: [[-0.99567159+0.j]
 [-0.09281684+0.j]
 [-0.00432621+0.j]
 [-0.00209615+0.j]], Best lowest eigenvalue: (7.638334409421077e-14+0j), First excited state eigenvalue: (9.002123036682667+0j)


### Print the results

In [37]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_2)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_2)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_2)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[-0.9956716 +0.j],
       [-0.09281682+0.j],
       [-0.00432616+0.j],
       [-0.00209605+0.j]]), 2: array([[0.99567159+0.j],
       [0.0928169 +0.j],
       [0.00432621+0.j],
       [0.00209614+0.j]]), 3: array([[-0.9956716 +0.j],
       [-0.09281678+0.j],
       [-0.00432623+0.j],
       [-0.00209616+0.j]]), 4: array([[0.99567159+0.j],
       [0.09281689+0.j],
       [0.00432617+0.j],
       [0.0020961 +0.j]]), 5: array([[0.99567159+0.j],
       [0.09281687+0.j],
       [0.00432622+0.j],
       [0.00209616+0.j]]), 6: array([[-0.9956716 +0.j],
       [-0.09281679+0.j],
       [-0.0043262 +0.j],
       [-0.00209612+0.j]])}, 'P_BFGS-TwoLocal': {1: array([[-0.9956716 +0.j],
       [-0.09281683+0.j],
       [-0.00432621+0.j],
       [-0.00209614+0.j]]), 2: array([[0.99567159+0.j],
       [0.09281689+0.j],
       [0.00432624+0.j],
       [0.00209616+0.j]]), 3: array([[0.9956716 +0.j],
       [0.09281682+0.j],
       [0.00432617+0.j],

### Computing \langle Q \rangle and the relative errors in \lambda_1

In [44]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_2.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_2.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-1.01942786e-07]]), array([[1.55084501e-07]]), array([[-1.67007667e-07]]), array([[9.61026978e-08]]), array([[8.94098576e-08]]), array([[-1.37727083e-07]])]

The expectation values for TwoLocal are:
[array([[-2.23637969e-08]]), array([[1.35500655e-07]]), array([[-9.77641219e-08]]), array([[3.58402037e-08]]), array([[9.85955215e-08]]), array([[-4.64963783e-08]])]

The expectation values for EfficientSU2 are:
[array([[-1.01942786e-07]]), array([[1.76686334e-07]]), array([[1.00525226e-07]]), array([[-2.47003525e-07]]), array([[2.09029819e-07]]), array([[2.63982277e-09]])]

The relative errors for RealAmplitudes are:
[5.919793027139756e-16, 1.602290646012494e-13, 2.1765105696450504e-13, 1.2431565356993488e-14, 6.235515321920543e-14, 1.4404829699373406e-14]

The relative errors for TwoLocal are:
[1.1346269968684533e-13, 8.031185873486269e-14, 2.229788706889308e-14, 1.4602156133611398e-13, 6.215782678496744e-14, 3.255886164926866e-13]



### For 3 qubits

### Get the matrix and the zeromode

In [52]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(3, 1.1)

The Schlogl operator matrix is:
[[  -0.275         2.95          0.            0.            0.
     0.            0.            0.        ]
 [   0.275        -3.225         5.9           0.            0.
     0.            0.            0.        ]
 [   0.            0.275       -11.62954545   11.82520661    0.
     0.            0.            0.        ]
 [   0.            0.            5.72954545  -28.46384298   23.70082645
     0.            0.            0.        ]
 [   0.            0.            0.           16.63863636  -56.70309917
    44.50206612    0.            0.        ]
 [   0.            0.            0.            0.           33.00227273
   -99.32252066   77.20413223    0.        ]
 [   0.            0.            0.            0.            0.
    54.82045455 -159.29731405  124.7822314 ]
 [   0.            0.            0.            0.            0.
     0.           82.09318182 -124.7822314 ]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix

### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [54]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_3, metrics_N_3, eigenvalues_N_3 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99999151]], Function calls = [1295 1505],             Best zeromode: [[ 9.95791140e-01+0.j]
 [ 9.15968164e-02+0.j]
 [ 3.06683995e-03+0.j]
 [ 7.63617912e-04+0.j]
 [ 1.97696875e-04+0.j]
 [ 1.81830007e-05+0.j]
 [-7.95797125e-06+0.j]
 [-1.35106777e-06+0.j]], Best lowest eigenvalue: (5.433068190541235e-05+0j), First excited state eigenvalue: (15.764960020835474+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[0.9999998]], Function calls = [5024 5024],             Best zeromode: [[9.95650237e-01+0.j]
 [9.30018673e-02+0.j]
 [4.52659362e-03+0.j]
 [2.30669497e-03+0.j]
 [1.66030373e-03+0.j]
 [1.24617459e-03+0.j]
 [8.90165018e-04+0.j]
 [5.87187951e-04+0.j]], Best lowest eigenvalue: (1.0654935067577753e-06+0j), First excited state eigenvalue: (9.025296646856633+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [2489 4655],             Best zeromode: [[-9.95669720e-01+0.j]
 [-9.28141326e-02+0.j]
 [-4.32733202e-03+0.j]
 [-2.09697440e-03+0.j]
 [-1.47236283e-03+0.j]
 [-1.09193427e-03+0.j]
 [-7.75346050e-04+0.j]
 [-5.10080121e-04+0.j]], Best lowest eigenvalue: (3.0831870390102267e-10+0j), First excited state eigenvalue: (4.502626072362091+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [4730 5016],             Best zeromode: [[9.95669365e-01+0.j]
 [9.28179027e-02+0.j]
 [4.32735955e-03+0.j]
 [2.09769593e-03+0.j]
 [1.47307984e-03+0.j]
 [1.09256357e-03+0.j]
 [7.75852747e-04+0.j]
 [5.10441136e-04+0.j]], Best lowest eigenvalue: (7.503331289626658e-11+0j), First excited state eigenvalue: (7.9575997458933365+0j)


### Print the results

In [60]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_3)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_3)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_3)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 9.95791140e-01+0.j],
       [ 9.15968164e-02+0.j],
       [ 3.06683995e-03+0.j],
       [ 7.63617912e-04+0.j],
       [ 1.97696875e-04+0.j],
       [ 1.81830007e-05+0.j],
       [-7.95797125e-06+0.j],
       [-1.35106777e-06+0.j]]), 2: array([[9.95731331e-01+0.j],
       [9.22087250e-02+0.j],
       [3.70352673e-03+0.j],
       [1.41157171e-03+0.j],
       [7.86931284e-04+0.j],
       [4.79713141e-04+0.j],
       [2.88223990e-04+0.j],
       [1.64420271e-04+0.j]]), 3: array([[9.95680368e-01+0.j],
       [9.27095014e-02+0.j],
       [4.22119713e-03+0.j],
       [2.00222555e-03+0.j],
       [1.38864897e-03+0.j],
       [1.02312519e-03+0.j],
       [7.23960488e-04+0.j],
       [4.75502273e-04+0.j]]), 4: array([[9.95669512e-01+0.j],
       [9.28160837e-02+0.j],
       [4.32911317e-03+0.j],
       [2.10038104e-03+0.j],
       [1.47600731e-03+0.j],
       [1.09528778e-03+0.j],
       [7.77979334e-04+0.j],
       [5.11892776e-04+0.j]])

### Computing \langle Q \rangle and relative errors in \lambda_1

In [63]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_3.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_3.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-0.00393734]]), array([[-0.00196205]]), array([[-0.00034189]]), array([[1.44947926e-07]]), array([[-6.82834528e-06]]), array([[5.45297107e-06]])]

The expectation values for TwoLocal are:
[array([[-0.00402858]]), array([[2.78240597e-05]]), array([[-1.33439921e-06]]), array([[1.19273312e-05]]), array([[-3.16885478e-06]]), array([[-2.68152197e-06]])]

The expectation values for EfficientSU2 are:
[array([[-0.00367521]]), array([[1.8195659e-05]]), array([[8.28070505e-06]]), array([[0.00060217]]), array([[-6.09139538e-06]]), array([[3.98679111e-06]])]

The relative errors for RealAmplitudes are:
[2.5012812421211317, 5.515467716188154, 0.00025674889828227165, 3.485704257799407e-08, 1.7764677712835586e-06, 4.8316540032119004e-11]

The relative errors for TwoLocal are:
[1.6848956251852107, 5.326318551087695e-09, 1.1478234275185339e-07, 1.2407766620105762e-09, 6.914340077824637e-10, 5.517187179882044e-08]

The relative errors for Efficient

### For V = 5.5

### For 2 qubits

### Get the matrix, zeromode, and eigenvalues

In [18]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(2, 5.5)

The Schlogl operator matrix is:
[[-1.375       2.95        0.          0.        ]
 [ 1.375      -4.325       5.9         0.        ]
 [ 0.          1.375      -8.36590909  8.96900826]
 [ 0.          0.          2.46590909 -8.96900826]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[  3.78125    -10.003125     8.1125       0.        ]
 [-10.003125    29.29875    -37.020625    12.33238636]
 [  8.1125     -37.020625   110.87914256 -97.15066679]
 [  0.          12.33238636 -97.15066679 160.8862185 ]]

The lowest two eigenvalues are:
[-4.257371493713317e-15, 9.1459297986593]

the available eigenvalues are: 
[ 4.44089210e-16+0.j -2.84354832e+00+0.j -6.37631943e+00+0.j
 -1.38150496e+01+0.j]

The minimum eigenvalue located is: 
(4.440892098500626e-16+0j)

The minimum zeromode located is: 
[[-0.90169082]
 [-0.42027962]
 [-0.09794652]
 [-0.02692909]]

The lowest two eigenvalues are: 
[-4.25737149e-15  9.14592980e+00]



### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [21]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2, eigenvalues_N_2 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = [130 220],             Best zeromode: [[-0.90169082+0.j]
 [-0.42027963+0.j]
 [-0.09794651+0.j]
 [-0.02692908+0.j]], Best lowest eigenvalue: (2.6645352591003757e-14+0j), First excited state eigenvalue: (36.83849256719023+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = [440 506],             Best zeromode: [[0.90169084+0.j]
 [0.4202796 +0.j]
 [0.0979465 +0.j]
 [0.0269291 +0.j]], Best lowest eigenvalue: (6.084022174945858e-14+0j), First excited state eigenvalue: (9.1459297986593+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [169 286],             Best zeromode: [[0.90169081+0.j]
 [0.42027965+0.j]
 [0.09794653+0.j]
 [0.02692909+0.j]], Best lowest eigenvalue: (1.687538997430238e-14+0j), First excited state eigenvalue: (9.145929798659312+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [255 375],             Best zeromode: [[-0.90169085+0.j]
 [-0.42027957+0.j]
 [-0.09794652+0.j]
 [-0.0269291 +0.j]], Best lowest eigenvalue: (1.1013412404281553e-13+0j), First excited state eigenvalue: (9.145929798659438+0j)


### Print the results

In [24]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_2)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_2)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_2)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[-0.90169082+0.j],
       [-0.42027963+0.j],
       [-0.09794651+0.j],
       [-0.02692908+0.j]]), 2: array([[-0.90169082+0.j],
       [-0.42027963+0.j],
       [-0.09794652+0.j],
       [-0.02692908+0.j]]), 3: array([[0.90169083+0.j],
       [0.42027962+0.j],
       [0.09794649+0.j],
       [0.02692907+0.j]]), 4: array([[-0.90169083+0.j],
       [-0.42027961+0.j],
       [-0.09794652+0.j],
       [-0.02692909+0.j]]), 5: array([[0.90169083+0.j],
       [0.42027961+0.j],
       [0.09794649+0.j],
       [0.02692907+0.j]]), 6: array([[-0.90169082+0.j],
       [-0.42027963+0.j],
       [-0.09794652+0.j],
       [-0.02692909+0.j]])}, 'P_BFGS-TwoLocal': {1: array([[-0.90169082+0.j],
       [-0.42027962+0.j],
       [-0.09794652+0.j],
       [-0.02692909+0.j]]), 2: array([[-0.90169083+0.j],
       [-0.42027961+0.j],
       [-0.09794652+0.j],
       [-0.0269291 +0.j]]), 3: array([[0.90169084+0.j],
       [0.4202796 +0.j],
       [0.097946

### Computing \langle Q \rangle and the relative errors in \lambda_1

In [27]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_2.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_2.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-1.3159443e-08]]), array([[-5.35912641e-09]]), array([[-6.35677962e-08]]), array([[-2.0718232e-08]]), array([[-8.20702578e-08]]), array([[2.57157443e-09]])]

The expectation values for TwoLocal are:
[array([[4.37160805e-09]]), array([[-4.81296079e-09]]), array([[-6.51527677e-08]]), array([[8.96527886e-08]]), array([[-5.94992129e-08]]), array([[3.14164029e-10]])]

The expectation values for EfficientSU2 are:
[array([[-1.3159443e-08]]), array([[-1.62462849e-07]]), array([[-6.72538738e-08]]), array([[-5.53491557e-08]]), array([[4.80653133e-08]]), array([[-5.60191577e-08]])]

The relative errors for RealAmplitudes are:
[3.027856475849003, 5.438265173487129e-15, 2.7968220892219517e-14, 5.671333680922291e-14, 2.6802878355043706e-14, 6.603607710662942e-15]

The relative errors for TwoLocal are:
[2.757977337982758e-14, 8.351621516426662e-15, 2.7171903491816047e-13, 8.914870409394971e-14, 1.942237561959689e-16, 2.7191325867435644e-15]

The

### For 3 qubits

### Get the matrix, zeromode, and eigenvalues

In [16]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(3, 5.5)

The Schlogl operator matrix is:
[[ -1.375        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  1.375       -4.325        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           1.375       -8.36590909   8.96900826   0.
    0.           0.           0.        ]
 [  0.           0.           2.46590909 -13.61673554  12.27603306
    0.           0.           0.        ]
 [  0.           0.           0.           4.64772727 -20.1964876
   15.94008264   0.           0.        ]
 [  0.           0.           0.           0.           7.92045455
  -28.22417355  20.08016529   0.        ]
 [  0.           0.           0.           0.           0.
   12.28409091 -37.81880165  24.81528926]
 [  0.           0.           0.           0.           0.
    0.          17.73863636 -24.81528926]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[    3.78125      -10.003125       8.1125         0.
      

### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [19]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_3, metrics_N_3, eigenvalues_N_3 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99999059]], Function calls = [ 651 1218],             Best zeromode: [[9.02216444e-01+0.j]
 [4.19371643e-01+0.j]
 [9.67588808e-02+0.j]
 [2.57912841e-02+0.j]
 [9.13747895e-03+0.j]
 [4.22838991e-03+0.j]
 [1.89807551e-03+0.j]
 [6.82422714e-04+0.j]], Best lowest eigenvalue: (0.0003939741664282792+0j), First excited state eigenvalue: (12.747876366961705+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = [1648 4256],             Best zeromode: [[0.90162587+0.j]
 [0.42024928+0.j]
 [0.09793949+0.j]
 [0.02692727+0.j]
 [0.01019478+0.j]
 [0.00506571+0.j]
 [0.00309899+0.j]
 [0.00221524+0.j]], Best lowest eigenvalue: (7.958078640513122e-13+0j), First excited state eigenvalue: (4.705671303254611+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [1710 3287],             Best zeromode: [[0.90162609+0.j]
 [0.42024895+0.j]
 [0.09793911+0.j]
 [0.02692689+0.j]
 [0.01019442+0.j]
 [0.0050654 +0.j]
 [0.00309873+0.j]
 [0.00221502+0.j]], Best lowest eigenvalue: (3.325340003357269e-12+0j), First excited state eigenvalue: (4.705671303010718+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [1980 5060],             Best zeromode: [[-0.90162577+0.j]
 [-0.4202495 +0.j]
 [-0.09793952+0.j]
 [-0.02692718+0.j]
 [-0.01019469+0.j]
 [-0.00506561+0.j]
 [-0.00309886+0.j]
 [-0.00221514+0.j]], Best lowest eigenvalue: (6.821210263296962e-13+0j), First excited state eigenvalue: (4.705671303220782+0j)


### Print the results

In [22]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_3)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_3)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_3)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[9.02216444e-01+0.j],
       [4.19371643e-01+0.j],
       [9.67588808e-02+0.j],
       [2.57912841e-02+0.j],
       [9.13747895e-03+0.j],
       [4.22838991e-03+0.j],
       [1.89807551e-03+0.j],
       [6.82422714e-04+0.j]]), 2: array([[0.90162573+0.j],
       [0.42024927+0.j],
       [0.0979406 +0.j],
       [0.02692808+0.j],
       [0.01019524+0.j],
       [0.00506581+0.j],
       [0.00309868+0.j],
       [0.00221462+0.j]]), 3: array([[0.90162467+0.j],
       [0.42025102+0.j],
       [0.09794198+0.j],
       [0.0269297 +0.j],
       [0.01019721+0.j],
       [0.00506818+0.j],
       [0.00310116+0.j],
       [0.00221704+0.j]]), 4: array([[-0.90162592+0.j],
       [-0.42024923+0.j],
       [-0.09793939+0.j],
       [-0.02692704+0.j],
       [-0.01019462+0.j],
       [-0.00506566+0.j],
       [-0.00309899+0.j],
       [-0.00221527+0.j]]), 5: array([[-0.90162584+0.j],
       [-0.42024937+0.j],
       [-0.09793949+0.j],
       [-0.02

### Computing \langle Q \rangle relative errors in \lambda_1

In [25]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_3.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_3.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-0.00422166]]), array([[2.57925068e-06]]), array([[8.8509101e-06]]), array([[-3.34822042e-07]]), array([[1.04552065e-07]]), array([[1.08071233e-07]])]

The expectation values for TwoLocal are:
[array([[-0.00189429]]), array([[-9.45902625e-07]]), array([[3.9331548e-07]]), array([[-3.61716521e-07]]), array([[2.12758416e-07]]), array([[3.13665345e-07]])]

The expectation values for EfficientSU2 are:
[array([[-0.00189273]]), array([[4.93034982e-07]]), array([[4.47926336e-07]]), array([[6.8669179e-08]]), array([[-1.34139679e-06]]), array([[3.17802587e-07]])]

The relative errors for RealAmplitudes are:
[1.70904522354866, 1.420529469359012, 2.0936314609510877e-11, 3.1087491245196665e-10, 1.8685702785057807e-10, 0.21760395818514705]

The relative errors for TwoLocal are:
[1.4898931242452222, 1.730936418862106e-11, 1.4531017488189229e-11, 4.238847307311687e-11, 9.771588792666873e-12, 1.569671512554432e-11]

The relative errors for Efficie

### For V = 10.5

### For 2 qubits

### Get the matrix, zeromode, and eigenvalues

In [16]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(2, 10.5)

The Schlogl operator matrix is:
[[-2.625       2.95        0.          0.        ]
 [ 2.625      -5.575       5.9         0.        ]
 [ 0.          2.625      -9.09642857  8.88265306]
 [ 0.          0.          3.19642857 -8.88265306]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.        ]
 [ -22.378125     46.67375     -56.770625     23.31696429]
 [  15.4875      -56.770625    127.77216837 -109.19318513]
 [   0.           23.31696429 -109.19318513  157.80305081]]

The lowest two eigenvalues are:
[-4.3517946982977e-15, 10.364814601313393]

the available eigenvalues are: 
[-1.52577371e+01+0.j -7.74424342e+00+0.j -9.70174270e-16+0.j
 -3.17710106e+00+0.j]

The minimum eigenvalue located is: 
(-9.701742699678762e-16+0j)

The minimum zeromode located is: 
[[-0.71268205]
 [-0.63416623]
 [-0.28215023]
 [-0.10153195]]

The lowest two eigenvalues are: 
[-4.35179470e-15  1.03648146e+01]



### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [19]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2, eigenvalues_N_2 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = [150 175],             Best zeromode: [[0.71268207+0.j]
 [0.63416621+0.j]
 [0.28215022+0.j]
 [0.10153194+0.j]], Best lowest eigenvalue: (6.667583152264456e-14+0j), First excited state eigenvalue: (10.36481460131371+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = [165 462],             Best zeromode: [[-0.71268204+0.j]
 [-0.63416623+0.j]
 [-0.28215024+0.j]
 [-0.10153194+0.j]], Best lowest eigenvalue: (4.33438007707565e-14+0j), First excited state eigenvalue: (10.364814601316427+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [169 364],             Best zeromode: [[-0.71268204+0.j]
 [-0.63416623+0.j]
 [-0.28215024+0.j]
 [-0.10153196+0.j]], Best lowest eigenvalue: (1.777744618181032e-14+0j), First excited state eigenvalue: (10.364814601313386+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [240 885],             Best zeromode: [[-0.71268202+0.j]
 [-0.63416625+0.j]
 [-0.28215026+0.j]
 [-0.10153197+0.j]], Best lowest eigenvalue: (2.2364055052292997e-14+0j), First excited state eigenvalue: (10.364814601313348+0j)


### Print the results

In [22]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_2)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_2)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_2)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.71268207+0.j],
       [0.63416621+0.j],
       [0.28215022+0.j],
       [0.10153194+0.j]]), 2: array([[0.71268204+0.j],
       [0.63416623+0.j],
       [0.28215025+0.j],
       [0.10153193+0.j]]), 3: array([[-0.71268203+0.j],
       [-0.63416624+0.j],
       [-0.28215026+0.j],
       [-0.10153193+0.j]]), 4: array([[0.71268204+0.j],
       [0.63416624+0.j],
       [0.28215023+0.j],
       [0.10153194+0.j]]), 5: array([[-0.71268204+0.j],
       [-0.63416624+0.j],
       [-0.28215023+0.j],
       [-0.10153193+0.j]]), 6: array([[-0.71268203+0.j],
       [-0.63416624+0.j],
       [-0.28215026+0.j],
       [-0.10153197+0.j]])}, 'P_BFGS-TwoLocal': {1: array([[-0.71268208+0.j],
       [-0.6341662 +0.j],
       [-0.28215022+0.j],
       [-0.10153194+0.j]]), 2: array([[-0.71268205+0.j],
       [-0.63416622+0.j],
       [-0.28215023+0.j],
       [-0.10153195+0.j]]), 3: array([[0.71268206+0.j],
       [0.63416621+0.j],
       [0.28215022+0

### Computing \langle Q \rangle and relative errors in \lambda_1

In [25]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_2.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_2.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-8.63430482e-09]]), array([[5.81854429e-09]]), array([[1.38849029e-08]]), array([[-1.78212567e-08]]), array([[-3.0925317e-08]]), array([[8.28371982e-08]])]

The expectation values for TwoLocal are:
[array([[-1.67120046e-08]]), array([[2.12010177e-08]]), array([[2.34364797e-08]]), array([[-1.48035178e-08]]), array([[2.9767279e-09]]), array([[-5.15238753e-09]])]

The expectation values for EfficientSU2 are:
[array([[-8.63430482e-09]]), array([[-2.15178342e-07]]), array([[-1.65411931e-08]]), array([[-1.04773453e-09]]), array([[3.51773591e-08]]), array([[6.67973133e-08]])]

The relative errors for RealAmplitudes are:
[3.067762294680631e-14, 5.141501052537371e-16, 9.254701894567267e-15, 7.352346505128439e-14, 1.0814290547170269e-13, 2.2279837894328604e-15]

The relative errors for TwoLocal are:
[2.0051854104895744e-14, 1.3196519368179251e-14, 4.284584210447809e-15, 1.1311302315582215e-14, 2.399367157850773e-15, 8.226401684059793e-14]



### For 3 qubits

### Get the matrix, zeromode, and eigenvalues

In [16]:
## Get everything
Q, hermitian_matrix, zeromode_classic, lowest_eigenvalues = get_zeromode(3, 10.5)

The Schlogl operator matrix is:
[[ -2.625        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  2.625       -5.575        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           2.625       -9.09642857   8.88265306   0.
    0.           0.           0.        ]
 [  0.           0.           3.19642857 -13.22193878  11.93061224
    0.           0.           0.        ]
 [  0.           0.           0.           4.33928571 -17.98418367
   15.07653061   0.           0.        ]
 [  0.           0.           0.           0.           6.05357143
  -23.41581633  18.35306122   0.        ]
 [  0.           0.           0.           0.           0.
    8.33928571 -29.5494898   21.79285714]
 [  0.           0.           0.           0.           0.
    0.          11.19642857 -21.79285714]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.         

### Run VQD for multiple ansatz depths (fix at 5000 optimization iterations)

In [19]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_3, metrics_N_3, eigenvalues_N_3 = analyze_ansatz_performance(hermitian_matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99869706]], Function calls = [826 546],             Best zeromode: [[0.72882081+0.j]
 [0.62494452+0.j]
 [0.26303023+0.j]
 [0.08506524+0.j]
 [0.02937499+0.j]
 [0.0224812 +0.j]
 [0.01740403+0.j]
 [0.01312949+0.j]], Best lowest eigenvalue: (0.052789330371233234+0j), First excited state eigenvalue: (38.68065023512601+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = [1552 1968],             Best zeromode: [[0.71209818+0.j]
 [0.6336465 +0.j]
 [0.28191885+0.j]
 [0.10144863+0.j]
 [0.03689792+0.j]
 [0.01481537+0.j]
 [0.00673183+0.j]
 [0.00345857+0.j]], Best lowest eigenvalue: (9.903189379656396e-13+0j), First excited state eigenvalue: (5.125516476732163+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = [2128 3154],             Best zeromode: [[0.71209792+0.j]
 [0.6336466 +0.j]
 [0.28191918+0.j]
 [0.10144883+0.j]
 [0.03689801+0.j]
 [0.01481542+0.j]
 [0.00673185+0.j]
 [0.00345859+0.j]], Best lowest eigenvalue: (8.255618411112664e-13+0j), First excited state eigenvalue: (5.125516475959007+0j)


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current proces

P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = [2046 5060],             Best zeromode: [[-0.71209806+0.j]
 [-0.63364655+0.j]
 [-0.28191901+0.j]
 [-0.10144872+0.j]
 [-0.03689794+0.j]
 [-0.01481536+0.j]
 [-0.00673182+0.j]
 [-0.00345858+0.j]], Best lowest eigenvalue: (-1.7053025658242404e-13+0j), First excited state eigenvalue: (5.125516476174697+0j)


### Print the results

In [22]:
# Print the zeromodes
print('The zeromodes are:')
print(zeromodes_N_3)
print()

# Print the performance metrics
print('The performance metrics are:')
print(metrics_N_3)
print()

# Print the eigenvalues
print('The eigenvalues are:')
print(eigenvalues_N_3)
print()

The zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.72882081+0.j],
       [0.62494452+0.j],
       [0.26303023+0.j],
       [0.08506524+0.j],
       [0.02937499+0.j],
       [0.0224812 +0.j],
       [0.01740403+0.j],
       [0.01312949+0.j]]), 2: array([[0.71209912+0.j],
       [0.63364639+0.j],
       [0.28191775+0.j],
       [0.10144701+0.j],
       [0.03689596+0.j],
       [0.0148132 +0.j],
       [0.00672955+0.j],
       [0.00345668+0.j]]), 3: array([[-0.71209818+0.j],
       [-0.63364649+0.j],
       [-0.28191889+0.j],
       [-0.10144865+0.j],
       [-0.03689792+0.j],
       [-0.01481533+0.j],
       [-0.00673178+0.j],
       [-0.00345854+0.j]]), 4: array([[-0.712098  +0.j],
       [-0.63364658+0.j],
       [-0.28191909+0.j],
       [-0.10144876+0.j],
       [-0.03689792+0.j],
       [-0.01481534+0.j],
       [-0.00673181+0.j],
       [-0.00345858+0.j]]), 5: array([[0.71209802+0.j],
       [0.63364661+0.j],
       [0.281919  +0.j],
       [0.10144864+0.j],
       [0.03689

### Computing \langle Q \rangle and relative errors in \lambda_1

In [25]:
# Initialize lists to store zeromodes and first excited state eigenvalues
zeromodes_list = []
first_excited_states = []

# Extract zeromodes and first excited states
for optimizer, zeromode_data in zeromodes_N_3.items():
    for index, zeromode_array in zeromode_data.items():
        zeromodes_list.append(zeromode_array.real)  # Extract real part of the zeromode arrays

for optimizer, eigenvalue_data in eigenvalues_N_3.items():
    for index, values in eigenvalue_data.items():
        first_excited_states.append(values['first excited state eigenvalue'].real)  # Extract real part of eigenvalues

# \lambda_1 (classical)
classical_first_excited_state = lowest_eigenvalues[1]

## Computing \langle Q \rangle and relative errors in \lambda_1
# Initialize
expectation_values = []
relative_errors = []

# Compute 
for zeromode in zeromodes_list:
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    expectation_value = get_expectation(Q, zeromode)
    expectation_values.append(expectation_value)

for eigenvalue in first_excited_states:
    error = np.abs(eigenvalue - classical_first_excited_state) / classical_first_excited_state
    relative_errors.append(error)

## Print the results
# For \langle Q \rangle
# For RealAmplitudes
print('The expectation values for RealAmplitudes are:')
print(expectation_values[:6])
print()

# For TwoLocal
print('The expectation values for TwoLocal are:')
print(expectation_values[6:12])
print()

# For EfficientSU2
print('The expectation values for EfficientSU2 are:')
print(expectation_values[12:18])
print()

# For relative errors
# For RealAmplitudes
print('The relative errors for RealAmplitudes are:')
print(relative_errors[:6])
print()

# For TwoLocal
print('The relative errors for TwoLocal are:')
print(relative_errors[6:12])
print()

# For EfficientSU2
print('The relative errors for EfficientSU2 are:')
print(relative_errors[12:18])
print()

The expectation values for RealAmplitudes are:
[array([[-0.05331512]]), array([[-6.52333184e-06]]), array([[-3.3621085e-07]]), array([[9.65284032e-08]]), array([[-2.74556616e-07]]), array([[6.1471186e-09]])]

The expectation values for TwoLocal are:
[array([[-0.05331227]]), array([[9.57952941e-07]]), array([[-4.08892856e-08]]), array([[4.34801141e-09]]), array([[6.98167291e-09]]), array([[-9.66825508e-08]])]

The expectation values for EfficientSU2 are:
[array([[-0.05331377]]), array([[4.33193428e-09]]), array([[9.21334286e-09]]), array([[-4.15423975e-07]]), array([[4.14756009e-07]]), array([[-4.62533294e-08]])]

The relative errors for RealAmplitudes are:
[6.54668342529864, 3.937693936439049, 1.6690180018595673e-11, 5.3266273889240867e-11, 8.819060900809764e-11, 4.633658101429132e-12]

The relative errors for TwoLocal are:
[1.5923566538198473, 1.6405644992322424e-11, 5.431395888853932e-11, 2.9174370903388504e-12, 7.62699430352662e-12, 2.4161910385739327e-11]

The relative errors for E